In [1]:
import numpy as np
from scipy . stats import truncnorm

In [4]:
#functions from exercise 7.2
def mutiplyGauss(m1 , s1 , m2 , s2):
    s = 1/(1/s1+1/s2)
    m = (m1/s1+m2/s2)*s
    return m, s


def divideGauss (m1 , s1 , m2 , s2):
    m, s = mutiplyGauss (m1 , s1 , m2 , -s2)
    return m, s

In [14]:
#given prior distributions for skills s1 and s2, as well as the result from the match between them,
#this function does all the neccessary message passing and returns the posteriors.
def messagepassing(prior_mean_s1, prior_var_s1, prior_mean_s2, prior_var_s2, y):

    var_t = 1
    
    #message from factor node to T:
    mu2_mean = prior_mean_s1 - prior_mean_s2
    mu2_var = prior_var_s1 + prior_var_s2 + var_t

    #effect of dirac depending on y 1/0:
    if y == 0:
        a, b = np.NINF, 0
    elif y == 1:
        a, b = 0, np.Inf

    #truncate gaussian
    a_scaled , b_scaled = (a - mu2_mean) / np.sqrt(mu2_var), (b - mu2_mean) / np.sqrt(mu2_var)
    trunc_m = truncnorm.mean(a_scaled , b_scaled , loc=mu2_mean , scale=np.sqrt(mu2_var))
    trunc_s = truncnorm.var(a_scaled , b_scaled , loc=mu2_mean , scale=np.sqrt(mu2_var))

    #Messages going back from t all the way back to s1, s2:

    mu5_mean, mu5_var = divideGauss(trunc_m, trunc_s, mu2_mean, mu2_var)

    mu6_1_mean = mu5_mean + prior_mean_s2
    mu6_2_mean = -mu5_mean + prior_mean_s1
    mu6_1_var = mu5_var + prior_var_s2 + var_t
    mu6_2_var = mu5_var + prior_var_s1 + var_t

    updated_mean_s1, updated_var_s1 = mutiplyGauss(prior_mean_s1, prior_var_s1, mu6_1_mean, mu6_1_var)
    updated_mean_s2, updated_var_s2 = mutiplyGauss(prior_mean_s2, prior_var_s2, mu6_2_mean, mu6_2_var)
    
    return updated_mean_s1, updated_var_s1, updated_mean_s2, updated_var_s2

In [20]:
S1_mean = 25
S1_var = 25/3
S2_mean = 25
S2_var = 25/3
y = 1

S1_mean_n, S1_var_n, S2_mean_n, S2_var_n = messagepassing(S1_mean, S1_var, S2_mean, S2_var, y)
print(S1_mean_n)
print(S1_var_n)
print(S2_mean_n)
print(S2_var_n)
print(S1_mean_n + S2_mean_n)


26.581909040411844
5.830897121196615
23.41809095958816
5.830897121196615
50.0
